<a href="https://colab.research.google.com/github/tomheston/fragility-metrics/blob/main/notebooks/pfi_rri_rq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Colab-ready: Exact UFI then PFI for a 2x2 table using Pearson's chi-square (no Yates)
# Inputs via prompts. Prints each output on its own line.

import numpy as np
from scipy.stats import chi2_contingency

# ---------- Core utilities ----------

def p_value_pearson(a, b, c, d):
    tbl = np.array([[a, b],[c, d]], dtype=float)
    if np.any(tbl < 0) or not np.isfinite(tbl.sum()) or tbl.sum() == 0:
        return np.nan
    chi2, p, dof, exp = chi2_contingency(tbl, correction=False)
    return float(p)

def p_along_x(a, b, c, d, x):
    aa, bb, cc, dd = a + x, b - x, c - x, d + x
    if min(aa, bb, cc, dd) < 0:
        return np.nan
    return p_value_pearson(aa, bb, cc, dd)

def compute_ufi_pfi(a, b, c, d, alpha=0.05, tol=1e-10, max_iter=200):
    """
    Exact UFI then PFI for a 2x2 table with fixed margins along:
      (a, b, c, d) -> (a+x, b-x, c-x, d+x)
    UFI = minimal |x| that flips significance (two-sided Pearson chi-square, no Yates).
    PFI = |4x| / n.
    Returns dict with inputs and results.
    """
    a, b, c, d = map(float, (a, b, c, d))
    n = a + b + c + d
    if n <= 0:
        raise ValueError("Total n must be > 0.")
    if min(a, b, c, d) < 0:
        raise ValueError("All cells must be >= 0.")

    # --- New: RRI and RQ ---
    rri = abs(a*d - b*c) / n
    rq  = (4.0 * rri) / n

    # Feasible x to keep all cells >= 0
    x_min = -min(a, d)      # left bound (<= 0)
    x_max =  min(b, c)      # right bound (>= 0)

    # Original p and state
    p0 = p_value_pearson(a, b, c, d)
    if not np.isfinite(p0):
        raise ValueError("Invalid table for chi-square.")
    sig0 = (p0 <= alpha)

    # Independence table (unique along this path)
    r1, r2 = a + b, c + d
    c1, c2 = a + c, b + d
    a_exp = r1 * c1 / n
    x_ind = a_exp - a  # displacement that reaches independence

    # Target: f(x) = p(x) - alpha
    def f(x):
        return p_along_x(a, b, c, d, x) - alpha

    # Choose search direction
    if sig0:
        # Move toward independence to raise p
        lo, hi = (0.0, x_ind) if x_ind >= 0 else (x_ind, 0.0)
    else:
        # Move away from independence to lower p
        if x_ind >= 0:
            lo, hi = (0.0, x_min)  # x_min <= 0
        else:
            lo, hi = (0.0, x_max)  # x_max >= 0

    # Ensure ordering and clip to feasibility
    if lo > hi:
        lo, hi = hi, lo
    lo = max(lo, x_min)
    hi = min(hi, x_max)
    if lo == hi:
        return {
            "inputs": {"a": a, "b": b, "c": c, "d": d},
            "alpha": alpha,
            "p_original": p0,
            "x_signed": None,
            "UFI": None,
            "PFI_decimal": None,
            "PFI_percent": None,
            "p_at_flip": None,
            "RRI": rri,
            "RQ": rq
        }

    # Bracket sign change for f(x)
    flo, fhi = f(lo), f(hi)
    if not (np.isfinite(flo) and np.isfinite(fhi)) or flo * fhi > 0:
        # grid scan to find sub-interval with sign change
        GRID = 400
        xs = np.linspace(lo, hi, GRID + 1)
        fs = np.array([f(x) for x in xs])
        bracket_found = False
        for i in range(GRID):
            if np.isfinite(fs[i]) and np.isfinite(fs[i+1]) and fs[i] * fs[i+1] <= 0:
                lo, hi = xs[i], xs[i+1]
                flo, fhi = fs[i], fs[i+1]
                bracket_found = True
                break
        if not bracket_found:
            # No flip within feasibility
            return {
                "inputs": {"a": a, "b": b, "c": c, "d": d},
                "alpha": alpha,
                "p_original": p0,
                "x_signed": None,
                "UFI": None,
                "PFI_decimal": None,
                "PFI_percent": None,
                "p_at_flip": None,
                "RRI": rri,
                "RQ": rq
            }

    # Bisection
    for _ in range(max_iter):
        mid = 0.5 * (lo + hi)
        fmid = f(mid)
        if not np.isfinite(fmid):
            mid = np.nextafter(mid, hi)
            fmid = f(mid)
            if not np.isfinite(fmid):
                break
        if abs(hi - lo) < tol:
            x_star = mid
            break
        if flo == 0:
            x_star = lo
            break
        if fhi == 0:
            x_star = hi
            break
        if flo * fmid <= 0:
            hi, fhi = mid, fmid
        else:
            lo, flo = mid, fmid
    else:
        x_star = 0.5 * (lo + hi)

    p_flip = p_along_x(a, b, c, d, x_star)
    UFI = abs(x_star)
    PFI_decimal = (4.0 * UFI) / n
    PFI_percent = 100.0 * PFI_decimal

    return {
        "inputs": {"a": a, "b": b, "c": c, "d": d},
        "alpha": alpha,
        "p_original": p0,
        "x_signed": x_star,      # signed displacement
        "UFI": UFI,              # |x|
        "PFI_decimal": PFI_decimal,
        "PFI_percent": PFI_percent,
        "p_at_flip": p_flip,
        "RRI": rri,
        "RQ": rq
    }

# ---------- Interactive runner ----------

def run_once():
    print("Enter 2x2 counts (nonnegative). Press Enter to accept default in brackets.")
    def read_float(prompt, default=None):
        s = input(prompt).strip()
        if s == "" and default is not None:
            return float(default)
        return float(s)

    a = read_float("a [1]: ", 1)
    b = read_float("b [1]: ", 1)
    c = read_float("c [1]: ", 1)
    d = read_float("d [1]: ", 1)

    alpha_in = input("alpha [0.05]: ").strip()
    alpha = float(alpha_in) if alpha_in else 0.05

    res = compute_ufi_pfi(a, b, c, d, alpha=alpha)

    # Pretty print one field per line
    print("\n--- Results ---")
    print(f"Inputs: a={res['inputs']['a']:.6g}, b={res['inputs']['b']:.6g}, c={res['inputs']['c']:.6g}, d={res['inputs']['d']:.6g}")
    print(f"Alpha: {res['alpha']}")
    print(f"Original p-value: {res['p_original'] if res['p_original'] is None else f'{res['p_original']:.10f}'}")
    print(f"Signed UFI (x): {res['x_signed'] if res['x_signed'] is None else f'{res['x_signed']:.10f}'}")
    print(f"UFI (|x|): {res['UFI'] if res['UFI'] is None else f'{res['UFI']:.10f}'}")
    print(f"PFI (decimal): {res['PFI_decimal'] if res['PFI_decimal'] is None else f'{res['PFI_decimal']:.10f}'}")
    print(f"PFI (%): {res['PFI_percent'] if res['PFI_percent'] is None else f'{res['PFI_percent']:.6f}'}")
    print(f"RRI: {res['RRI'] if res['RRI'] is None else f'{res['RRI']:.10f}'}")
    print(f"RQ: {res['RQ'] if res['RQ'] is None else f'{res['RQ']:.10f}'}")
    print(f"p-value at flip: {res['p_at_flip'] if res['p_at_flip'] is None else f'{res['p_at_flip']:.10f}'}")
    print("----------------\n")

if __name__ == "__main__":
    while True:
        run_once()
        again = input("Run another? [y/N]: ").strip().lower()
        if again != "y":
            break


Enter 2x2 counts (nonnegative). Press Enter to accept default in brackets.
a [1]: 7
b [1]: 13
c [1]: 14
d [1]: 12
alpha [0.05]: 

--- Results ---
Inputs: a=7, b=13, c=14, d=12
Alpha: 0.05
Original p-value: 0.2033350778
Signed UFI (x): -1.1519740078
UFI (|x|): 1.1519740078
PFI (decimal): 0.1001716529
PFI (%): 10.017165
RRI: 2.1304347826
RQ: 0.1852551985
p-value at flip: 0.0500000000
----------------

Run another? [y/N]: n
